In [13]:
import os, sys, email
import numpy as np
import pandas as pd

from fileRetrieval import path_list
from fileToDataFrame import fileToDataFrame

In [14]:
# ディレクトリ 内のメールファイルを読み込む

directory_path = "enron/maildir/allen-p/"

file_list, dir_list=path_list(directory_path)

In [16]:
# テキストファイルをデータフレームに格納する

df = fileToDataFrame(file_list)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3034 entries, 0 to 3033
Data columns (total 13 columns):
Path          3034 non-null object
Message-ID    3034 non-null object
Date          3034 non-null object
From          3034 non-null object
To            2971 non-null object
Cc            177 non-null object
Bcc           177 non-null object
X-From        3034 non-null object
X-To          3034 non-null object
X-cc          3034 non-null object
X-bcc         3034 non-null object
Subject       3034 non-null object
Message       3034 non-null object
dtypes: object(13)
memory usage: 308.2+ KB
